In [ ]:
# Modified visualization script to correctly handle feature categories

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
from matplotlib.gridspec import GridSpec
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mtick

# Set the aesthetic style
plt.style.use('fivethirtyeight')
sns.set_palette("viridis")

# Define custom colors
colors = ["#4287f5", "#42c5f5", "#42f5d1", "#42f56f", 
          "#9ef542", "#f5ee42", "#f5c242", "#f54242"]
custom_cmap = LinearSegmentedColormap.from_list("custom_viridis", colors)

# Create directories if they don't exist
viz_dir = 'model_visualizations'
os.makedirs(viz_dir, exist_ok=True)

# Load the models and data
try:
    # Load the datasets
    print("Loading datasets...")
    data_path = 'task_estimation_results_improved/data_splits.pkl'
    
    with open(data_path, 'rb') as f:
        data = pickle.load(f)
        
    X_train = data.get('X_train')
    X_test = data.get('X_test')
    y_train = data.get('y_train')
    y_test = data.get('y_test')
    feature_names = data.get('feature_names')
    
    # Load the models
    models = {}
    model_files = [f for f in os.listdir('task_estimation_results_improved') if f.endswith('_model.pkl')]
    
    for model_file in model_files:
        model_name = model_file.replace('_model.pkl', '')
        with open(f'task_estimation_results_improved/{model_file}', 'rb') as f:
            models[model_name] = pickle.load(f)
    
    # Find the best model
    with open('task_estimation_results_improved/best_model.pkl', 'rb') as f:
        best_model = pickle.load(f)
        
    # Identify best model name
    best_model_name = None
    for name, model in models.items():
        if model == best_model:
            best_model_name = name
            break
    
    print(f"Loaded models: {list(models.keys())}")
    print(f"Best model: {best_model_name}")
    
except Exception as e:
    print(f"Error loading models or data: {e}")
    exit(1)

# Generate predictions for all models
predictions = {}
for name, model in models.items():
    predictions[name] = model.predict(X_test)

# Calculate ensemble prediction
ensemble_pred = np.mean([predictions[name] for name in models.keys()], axis=0)

# =============== 1. Feature Importance Visualization ===============
print("\nCreating feature importance visualizations...")

# Create a function to plot feature importance
def plot_feature_importance(model, feature_names, title, filename, top_n=20):
    """
    Plot feature importance for a model with enhanced styling.
    
    Parameters:
    -----------
    model : model object
        The trained model with feature_importances_ attribute
    feature_names : list
        List of feature names
    title : str
        Plot title
    filename : str
        Output filename
    top_n : int
        Number of top features to display
    """
    if not hasattr(model, 'feature_importances_'):
        print(f"Model {title} doesn't have feature_importances_ attribute")
        return None
    
    # Get feature importances and sort them
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    # Create DataFrames for visualization and analysis
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    # Print the top features for debugging
    print(f"\nTop 10 features for {title}:")
    for i, (feature, importance) in enumerate(zip(importance_df['Feature'].head(10), 
                                                importance_df['Importance'].head(10))):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    # Analyze feature prefixes to better understand what types we have
    feature_prefixes = set()
    for feature in importance_df['Feature'].head(top_n):
        parts = feature.split('__')
        if len(parts) > 1:
            feature_prefixes.add(parts[0])
    
    print(f"Feature prefixes found in top {top_n}: {feature_prefixes}")
    
    # Check if any count features are in the top features (modified to use 'count' rather than 'count_std')
    count_features = [f for f in importance_df['Feature'].head(top_n) if 'count' in f]
    if count_features:
        print(f"Found count-related features in top {top_n}: {count_features}")
    
    # Select top N features
    n_features = min(top_n, len(feature_names))
    top_indices = indices[:n_features]
    
    # Create the figure
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Plot horizontal bars for better readability with feature names
    colors = custom_cmap(np.linspace(0, 1, n_features))
    bars = ax.barh(range(n_features), importances[top_indices], color=colors, alpha=0.8)
    
    # Add feature names and importance values
    for i, bar in enumerate(bars):
        ax.text(
            bar.get_width() * 1.01, 
            bar.get_y() + bar.get_height()/2, 
            f"{importances[top_indices[i]]:.3f}", 
            va='center', 
            fontsize=10, 
            fontweight='bold'
        )
    
    # Customize the plot
    ax.set_yticks(range(n_features))
    ax.set_yticklabels([feature_names[i] for i in top_indices], fontsize=11)
    ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
    ax.set_xlabel('Feature Importance', fontsize=12, fontweight='bold')
    
    # Add a grid for better readability
    ax.xaxis.grid(True, linestyle='--', alpha=0.7)
    
    # Highlight count features if present (modified to use 'count' rather than 'count_std')
    for i, idx in enumerate(top_indices):
        feature_name = feature_names[idx]
        if 'count' in feature_name:
            # Add a highlight or outline to the bar
            ax.get_yticklabels()[i].set_color('darkblue')
            ax.get_yticklabels()[i].set_fontweight('bold')
            # Add a subtle background highlight
            ax.axhspan(i-0.4, i+0.4, color='lightblue', alpha=0.3)
    
    # Add styling
    fig.tight_layout()
    plt.savefig(f"{viz_dir}/{filename}", dpi=300, bbox_inches='tight')
    plt.close()
    
    return importance_df

# Plot feature importance for all models
importance_dfs = {}
for name, model in models.items():
    if hasattr(model, 'feature_importances_'):
        importance_df = plot_feature_importance(
            model, 
            feature_names,
            f"Feature Importance - {name}",
            f"{name}_feature_importance.png"
        )
        importance_dfs[name] = importance_df

# Create a special plot for the best model
if best_model_name and hasattr(models[best_model_name], 'feature_importances_'):
    best_importance_df = plot_feature_importance(
        models[best_model_name],
        feature_names,
        f"Feature Importance - Best Model ({best_model_name})",
        "best_model_feature_importance.png",
        top_n=20
    )
    
    # =============== 2. Feature Category Importance ===============
    print("Creating feature category importance visualization...")
    
    # Define feature categories based on actual prefixes in the data
    # Modify to match actual feature patterns in the dataset
    categories = {
        'Issue Type': [f for f in feature_names if 'type_' in f or 'is_type_' in f],
        'Priority': [f for f in feature_names if 'priority' in f],
        'Project Stats': [f for f in feature_names if ('count' in f and 'count_' not in f) or 'pct_minmax__' in f],
        'Team Size': [f for f in feature_names if 'team_size' in f],
        'Created Time': [f for f in feature_names if 'created_' in f],
        'Robust Stats': [f for f in feature_names if 'stat_robust' in f],
        'Other': []
    }
    
    # Assign remaining features to 'Other'
    for feature in feature_names:
        if not any(feature in cat_features for cat_features in categories.values()):
            categories['Other'].append(feature)
            
    # Print categorized features for verification
    print("\nFeatures by category:")
    for category, cat_features in categories.items():
        if cat_features:
            print(f"{category} ({len(cat_features)}): {', '.join(cat_features[:5])}" + 
                  ("..." if len(cat_features) > 5 else ""))
    
    # Calculate importance by category
    category_importance = {}
    for category, cat_features in categories.items():
        if cat_features:  # Skip empty categories
            total_importance = sum(best_importance_df.loc[best_importance_df['Feature'].isin(cat_features), 'Importance'])
            category_importance[category] = total_importance
            print(f"{category}: Total importance = {total_importance:.4f}")
    
    # Sort categories by importance
    categories_sorted = sorted(category_importance.items(), key=lambda x: x[1], reverse=True)
    categories_names = [item[0] for item in categories_sorted]
    categories_values = [item[1] for item in categories_sorted]
    
    # Create a pie chart for category importance
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 9))
    
    # Pie chart
    wedges, texts, autotexts = ax1.pie(
        categories_values, 
        labels=categories_names,
        autopct='%1.1f%%',
        startangle=90,
        colors=custom_cmap(np.linspace(0, 1, len(categories_values))),
        wedgeprops={'edgecolor': 'w', 'linewidth': 1},
        textprops={'fontsize': 11, 'fontweight': 'bold'}
    )
    
    # Customize pie chart
    for autotext in autotexts:
        autotext.set_fontsize(9)
        autotext.set_fontweight('bold')
    ax1.set_title('Feature Importance by Category', fontsize=16, fontweight='bold', pad=20)
    
    # Bar chart
    bars = ax2.bar(
        categories_names, 
        categories_values,
        color=custom_cmap(np.linspace(0, 1, len(categories_values))),
        alpha=0.8
    )
    
    # Add values on top of bars
    for bar in bars:
        height = bar.get_height()
        ax2.text(
            bar.get_x() + bar.get_width()/2.,
            height * 1.01,
            f'{height:.3f}',
            ha='center',
            va='bottom',
            fontsize=9,
            fontweight='bold'
        )
    
    # Customize bar chart
    ax2.set_ylabel('Total Importance', fontsize=12, fontweight='bold')
    ax2.set_title('Feature Importance by Category', fontsize=16, fontweight='bold', pad=20)
    ax2.set_xticklabels(categories_names, rotation=45, ha='right', fontsize=11)
    ax2.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"{viz_dir}/feature_importance_by_category.png", dpi=300, bbox_inches='tight')
    plt.close()

# =============== 3. Model Performance Comparison ===============
print("Creating model performance comparison visualizations...")

# Define a function to calculate metrics for all models
def calculate_metrics(y_true, predictions_dict):
    """Calculate MAE, RMSE, and R² for all models."""
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from scipy.stats import spearmanr
    
    metrics = {}
    for name, y_pred in predictions_dict.items():
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        r2 = r2_score(y_true, y_pred)
        spearman_corr, _ = spearmanr(y_true, y_pred)
        
        metrics[name] = {
            'MAE': mae,
            'RMSE': rmse,
            'R²': r2,
            'Spearman': spearman_corr
        }
    
    # Add ensemble metrics
    ensemble_pred = np.mean([predictions_dict[name] for name in predictions_dict.keys()], axis=0)
    mae = mean_absolute_error(y_true, ensemble_pred)
    rmse = np.sqrt(mean_squared_error(y_true, ensemble_pred))
    r2 = r2_score(y_true, ensemble_pred)
    spearman_corr, _ = spearmanr(y_true, ensemble_pred)
    
    metrics['Ensemble'] = {
        'MAE': mae,
        'RMSE': rmse,
        'R²': r2,
        'Spearman': spearman_corr
    }
    
    return metrics

# Calculate metrics
metrics = calculate_metrics(y_test, predictions)

# Create a DataFrame for the metrics
metrics_df = pd.DataFrame({
    model_name: {metric: value for metric, value in model_metrics.items()}
    for model_name, model_metrics in metrics.items()
})

# Transpose to make models as rows
metrics_df = metrics_df.T

# Create a comprehensive model performance comparison visualization
fig = plt.figure(figsize=(16, 12))
gs = GridSpec(2, 2, figure=fig)

# 1. MAE Comparison
ax1 = fig.add_subplot(gs[0, 0])
metrics_df.sort_values('MAE').plot(
    y='MAE', 
    kind='bar', 
    color=custom_cmap(np.linspace(0, 1, len(metrics_df))), 
    ax=ax1,
    alpha=0.8
)
ax1.set_title('Mean Absolute Error (Lower is Better)', fontsize=14, fontweight='bold')
ax1.set_ylabel('MAE (Hours)', fontsize=12)
for i, v in enumerate(metrics_df.sort_values('MAE')['MAE']):
    ax1.text(i, v + 0.1, f'{v:.2f}', ha='center', fontsize=10, fontweight='bold')
ax1.grid(True, linestyle='--', alpha=0.7)

# 2. R² Comparison
ax2 = fig.add_subplot(gs[0, 1])
metrics_df.sort_values('R²', ascending=False).plot(
    y='R²', 
    kind='bar', 
    color=custom_cmap(np.linspace(0, 1, len(metrics_df))), 
    ax=ax2,
    alpha=0.8
)
ax2.set_title('R² Score (Higher is Better)', fontsize=14, fontweight='bold')
ax2.set_ylabel('R²', fontsize=12)
for i, v in enumerate(metrics_df.sort_values('R²', ascending=False)['R²']):
    ax2.text(i, max(v - 0.05, 0.01), f'{v:.4f}', ha='center', fontsize=10, fontweight='bold')
ax2.grid(True, linestyle='--', alpha=0.7)

# 3. Comprehensive Metric Comparison
ax3 = fig.add_subplot(gs[1, :])
metrics_to_plot = ['MAE', 'RMSE', 'R²', 'Spearman']
# Normalize metrics for better visualization
normalized_metrics = metrics_df.copy()
for metric in metrics_to_plot:
    if metric in ['MAE', 'RMSE']:  # Lower is better
        min_val = normalized_metrics[metric].min()
        max_val = normalized_metrics[metric].max()
        if max_val > min_val:
            normalized_metrics[metric] = 1 - (normalized_metrics[metric] - min_val) / (max_val - min_val)
        else:
            normalized_metrics[metric] = 1
    else:  # Higher is better
        min_val = normalized_metrics[metric].min()
        max_val = normalized_metrics[metric].max()
        if max_val > min_val:
            normalized_metrics[metric] = (normalized_metrics[metric] - min_val) / (max_val - min_val)
        else:
            normalized_metrics[metric] = 1

normalized_metrics[metrics_to_plot].plot(
    kind='bar', 
    ax=ax3,
    figsize=(12, 6),
    colormap=custom_cmap,
    alpha=0.8
)
ax3.set_title('Normalized Performance Metrics (Higher is Better)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Normalized Score', fontsize=12)
ax3.grid(True, linestyle='--', alpha=0.7)
ax3.set_ylim(0, 1.1)
ax3.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))

# General title for the figure
fig.suptitle('Model Performance Comparison', fontsize=18, fontweight='bold', y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig(f"{viz_dir}/model_performance_comparison.png", dpi=300, bbox_inches='tight')
plt.close()

# Save the original metrics to CSV for reference
metrics_df.to_csv(f"{viz_dir}/model_performance_metrics.csv")
print(f"Saved performance metrics to {viz_dir}/model_performance_metrics.csv")

# =============== 4. Actual vs Predicted Visualizations ===============
print("Creating actual vs predicted visualizations...")

# Define a function to create enhanced actual vs predicted plots
def enhanced_actual_vs_predicted_plot(y_true, y_pred, title, filename):
    """Create an enhanced actual vs predicted plot with residuals and distribution."""
    # Calculate additional metrics
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    from scipy.stats import spearmanr
    
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    spearman, _ = spearmanr(y_true, y_pred)
    
    # Create figure with gridspec
    fig = plt.figure(figsize=(18, 10))
    gs = GridSpec(2, 3, figure=fig)
    
    # 1. Actual vs Predicted Scatter Plot
    ax1 = fig.add_subplot(gs[0, 0:2])
    scatter = ax1.scatter(
        y_true, 
        y_pred, 
        alpha=0.5, 
        c=np.abs(y_true - y_pred), 
        cmap='viridis', 
        edgecolors='w', 
        linewidth=0.5
    )
    ax1.plot([0, y_true.max()], [0, y_true.max()], 'r--', linewidth=2)
    ax1.set_xlabel('Actual Hours', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Predicted Hours', fontsize=12, fontweight='bold')
    ax1.set_title('Actual vs Predicted Hours', fontsize=14, fontweight='bold')
    
    # Add color bar to indicate prediction error
    cbar = plt.colorbar(scatter, ax=ax1)
    cbar.set_label('Absolute Error', fontsize=10, fontweight='bold')
    
    # Add metrics to the plot
    metrics_text = (
        f"MAE: {mae:.2f}\n"
        f"RMSE: {rmse:.2f}\n"
        f"R²: {r2:.4f}\n"
        f"Spearman ρ: {spearman:.4f}"
    )
    ax1.text(
        0.05, 0.95, metrics_text,
        transform=ax1.transAxes,
        fontsize=11,
        fontweight='bold',
        verticalalignment='top',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8)
    )
    ax1.grid(True, linestyle='--', alpha=0.7)
    
    # 2. Residuals Plot
    ax2 = fig.add_subplot(gs[1, 0:2])
    residuals = y_true - y_pred
    scatter = ax2.scatter(
        y_pred, 
        residuals, 
        alpha=0.5, 
        c=np.abs(residuals), 
        cmap='coolwarm', 
        edgecolors='w', 
        linewidth=0.5
    )
    ax2.axhline(y=0, color='r', linestyle='--', linewidth=2)
    ax2.set_xlabel('Predicted Hours', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Residuals (Actual - Predicted)', fontsize=12, fontweight='bold')
    ax2.set_title('Residual Plot', fontsize=14, fontweight='bold')
    
    # Add color bar for residuals
    cbar = plt.colorbar(scatter, ax=ax2)
    cbar.set_label('Absolute Residual', fontsize=10, fontweight='bold')
    
    # Find large residuals and add annotations
    large_residuals = np.abs(residuals) > np.percentile(np.abs(residuals), 95)
    if np.sum(large_residuals) > 0:
        for i, (x, y) in enumerate(zip(y_pred[large_residuals], residuals[large_residuals])):
            # Only label a few points to avoid cluttering
            if i % max(1, int(np.sum(large_residuals) / 5)) == 0:
                ax2.annotate(
                    f"{y:.0f}",
                    xy=(x, y),
                    xytext=(x+2, y),
                    fontsize=8,
                    arrowprops=dict(arrowstyle="->", color='black', alpha=0.6)
                )
    ax2.grid(True, linestyle='--', alpha=0.7)
    
    # 3. Error Distribution
    ax3 = fig.add_subplot(gs[0, 2])
    sns.histplot(residuals, kde=True, ax=ax3, color='skyblue', bins=20, alpha=0.7)
    ax3.axvline(x=0, color='r', linestyle='--', linewidth=2)
    ax3.set_xlabel('Residual Error', fontsize=12, fontweight='bold')
    ax3.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax3.set_title('Error Distribution', fontsize=14, fontweight='bold')
    
    # Add stats about the error distribution
    error_stats = (
        f"Mean Error: {np.mean(residuals):.2f}\n"
        f"Median Error: {np.median(residuals):.2f}\n"
        f"Std Dev: {np.std(residuals):.2f}"
    )
    ax3.text(
        0.05, 0.95, error_stats,
        transform=ax3.transAxes,
        fontsize=10,
        fontweight='bold',
        verticalalignment='top',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8)
    )
    ax3.grid(True, linestyle='--', alpha=0.7)
    
    # 4. Percent Error Analysis
    ax4 = fig.add_subplot(gs[1, 2])
    # Calculate percent error, handling division by zero
    percent_error = np.zeros_like(y_true, dtype=float)
    non_zero_mask = y_true != 0
    percent_error[non_zero_mask] = 100 * np.abs(residuals[non_zero_mask]) / y_true[non_zero_mask]
    
    # Plot histogram of percent errors
    sns.histplot(percent_error, kde=False, ax=ax4, color='lightgreen', bins=20, alpha=0.7)
    ax4.set_xlabel('Percent Error (%)', fontsize=12, fontweight='bold')
    ax4.set_ylabel('Frequency', fontsize=12, fontweight='bold')
    ax4.set_title('Percent Error Distribution', fontsize=14, fontweight='bold')
    
    # Compute percent error stats
    percent_stats = (
        f"Mean %Error: {np.mean(percent_error):.2f}%\n"
        f"Median %Error: {np.median(percent_error):.2f}%\n"
        f"% within 25% error: {100*np.sum(percent_error <= 25)/len(percent_error):.1f}%"
    )
    ax4.text(
        0.05, 0.95, percent_stats,
        transform=ax4.transAxes,
        fontsize=10,
        fontweight='bold',
        verticalalignment='top',
        bbox=dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.8)
    )
    ax4.grid(True, linestyle='--', alpha=0.7)
    
    # General title for the figure
    fig.suptitle(title, fontsize=18, fontweight='bold', y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.savefig(f"{viz_dir}/{filename}", dpi=300, bbox_inches='tight')
    plt.close()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
import pickle
import os
from scipy.stats import spearmanr
from matplotlib.gridspec import GridSpec
from matplotlib import cm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mtick

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
output_dir = 'task_estimation_hypertuned_100_without_count'
os.makedirs(output_dir, exist_ok=True)

# Load the dataset
data_path = 'merged_task_data/merged_project_task_data.csv'
df = pd.read_csv(data_path, low_memory=False)
print(f"Original dataset shape: {df.shape}")

# Convert string columns that should be numeric
for col in df.columns:
    if col.startswith('fields.issuetype') or col.startswith('fields.priority'):
        # Keep these as string/categorical
        continue
    try:
        # Try converting to numeric, coerce errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
    except:
        # If conversion fails completely, leave as is
        pass

# Use only 5% of the data for faster processing
df = df.sample(frac=1, random_state=42)
print(f"Reduced dataset shape (5%): {df.shape}")

# Define target variable - to be used only as the target, never as a feature
target_variable = 'total_resolution_hours'

# Define fields that must be excluded
fields_to_exclude = ['fields.status.name', 'fields.project.key', 'fields.priority.name', 'fields.project.name']

# Check which of these fields actually exist in the dataframe
existing_fields_to_exclude = [field for field in fields_to_exclude if field in df.columns]
print(f"\nFields to exclude that exist in the dataframe: {existing_fields_to_exclude}")

# Remove the fields from the dataframe itself to avoid correlation errors
if existing_fields_to_exclude:
    print(f"Dropping these fields from the dataframe: {existing_fields_to_exclude}")
    df = df.drop(columns=existing_fields_to_exclude)
    print(f"Dataframe shape after dropping: {df.shape}")

# Check for missing values in the target variable
print(f"Missing values in target: {df[target_variable].isna().sum()}")

# Drop rows with missing target values
df = df.dropna(subset=[target_variable])
print(f"Dataset shape after dropping missing targets: {df.shape}")

# Plot distribution of target variable
plt.figure(figsize=(10, 6))
sns.histplot(df[target_variable].clip(0, 500), bins=50)
plt.title('Distribution of Total Resolution Hours (clipped at 500 for visibility)')
plt.xlabel('Hours')
plt.ylabel('Count')
plt.savefig(f'{output_dir}/target_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# Identify numeric columns for correlation analysis
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
# Exclude target from potential features
numeric_features = [col for col in numeric_columns if col != target_variable]
print(f"\nUsing {len(numeric_features)} numeric features for correlation calculation")

# Calculate correlations between features and target
if target_variable in df.columns and numeric_features:
    # Create a dataframe with just the features and target for correlation
    corr_df = df[numeric_features + [target_variable]]
    target_correlations = corr_df.corr()[target_variable].drop(target_variable).sort_values(ascending=False)
    
    print("Top 10 correlated features with target:")
    print(target_correlations.head(10))
    print("\nBottom 10 correlated features with target:")
    print(target_correlations.tail(10))
    
    # Print all features that have correlation > 0.8 with target
    high_corr_features = target_correlations[target_correlations > 0.8].index.tolist()
    print(f"\nFeatures with >0.8 correlation with target: {high_corr_features}")
else:
    print("Cannot calculate target correlations - either target or features missing")
    high_corr_features = []
    
# Define suspicious features (features that may lead to data leakage)
suspicious_features = [
    # Resolution-related features that would leak the target information
    'avg_resolution_hours', 'median_resolution_hours', 
    'resolution_hours', 'log_resolution_hours'
]

# Add highly correlated features to suspicious list
suspicious_features.extend(high_corr_features)

# # Keep count_std__total_issues despite high correlation (if it exists)
# if 'count_std__total_issues' in suspicious_features:
#     suspicious_features.remove('count_std__total_issues')
#     print("Keeping 'count_std__total_issues' despite high correlation with target")

# Remove count_std__total_issues
if 'count_std__total_issues' in suspicious_features:
    # Do nothing, it's already excluded
    print("Removing 'count_std__total_issues' from features")
else:
    suspicious_features.append('count_std__total_issues')
    print("'count_std__total_issues' marked for exclusion")

# Add excluded fields to suspicious list
suspicious_features.extend(fields_to_exclude)

# Make sure target variable is not in suspicious features list (it's our target, not a feature)
if target_variable in suspicious_features:
    suspicious_features.remove(target_variable)
    
print(f"\nRemoving suspicious features: {suspicious_features}")

# Original features list, excluding suspicious features
features = [
    'fields.issuetype.id', 'fields.priority.id', 'priority_id', 'issue_type_id',
    'type_task', 'type_bug', 'inward_count', 'outward_count', 'type_sub_task',
    'created_hour', 'created_month', 'created_year',
    'is_type_bug', 'is_type_task', 'is_type_story', 'is_type_improvement',
    'is_type_new_feature', 'is_type_epic', 'is_type_sub-task',
    'is_priority_blocker', 'is_priority_critical', 'is_priority_major',
    'is_priority_minor', 'is_priority_trivial',
    'pct_minmax__type_bug_pct', 'pct_minmax__type_task_pct',
    'pct_minmax__type_new_feature_pct', 'pct_minmax__type_epic_pct',
    'pct_minmax__type_improvement_pct', 'pct_minmax__type_story_pct',
    'pct_minmax__type_documentation_pct', 'pct_minmax__priority_critical_pct',
    'pct_minmax__priority_blocker_pct', 'pct_minmax__priority_high_pct',
    'pct_minmax__priority_low_pct',
    'remainder__team_size_creators', 'remainder__team_size_assignees',
    'remainder__team_size_combined',
    'stat_robust__weighted_priority_score', 'stat_robust__issue_type_entropy',
    'stat_robust__high_to_low_priority_ratio', 'stat_robust__bug_ratio'
]

# If count_std__total_issues exists in the dataframe, add it to our features list
# if 'count_std__total_issues' in df.columns and 'count_std__total_issues' not in features:
#     features.append('count_std__total_issues')
#     print("Added 'count_std__total_issues' to features list")

# Ensure we exclude the additional fields by checking if they exist in the dataset
for field in fields_to_exclude:
    if field in df.columns and field not in suspicious_features:
        suspicious_features.append(field)
        print(f"Added {field} to suspicious features list")

# Filter dataset to include only features available in our dataset and not in suspicious list
available_features = [f for f in features if f in df.columns and f not in suspicious_features]
missing_features = [f for f in features if f not in df.columns]

print(f"Available features: {len(available_features)}")
print(f"Missing features: {len(missing_features)}")
if missing_features:
    print(f"Missing feature list: {missing_features}")
    
# Make absolutely sure we're not using the target as a feature
if target_variable in available_features:
    available_features.remove(target_variable)
    print(f"WARNING: Removed target variable '{target_variable}' from features list")
    
# Explicitly check all fields that need to be dropped
print("\nChecking for fields that need to be dropped:")
for field in fields_to_exclude:
    if field in df.columns:
        print(f"Field '{field}' exists in the dataframe")
        if field in available_features:
            available_features.remove(field)
            print(f"-> Removed '{field}' from available features")
    else:
        print(f"Field '{field}' does not exist in the dataframe")

# Double-check if any of these fields might be in the final feature list
for field in fields_to_exclude:
    if field in available_features:
        available_features.remove(field)
        print(f"-> Explicitly removed '{field}' from available features")
        
# Print final confirmation
print("\nConfirming excluded fields:")
for field in fields_to_exclude:
    print(f"'{field}' is {'NOT' if field not in available_features else 'STILL'} in the feature list")

# Check for multicollinearity
# Use only numeric features for correlation calculation
numeric_available_features = [f for f in available_features if f in numeric_features]
print(f"\nUsing {len(numeric_available_features)} numeric features for multicollinearity check")

if len(numeric_available_features) > 0:
    # Verify one more time we don't have the target as a feature
    if target_variable in numeric_available_features:
        numeric_available_features.remove(target_variable)
        print(f"WARNING: Removed target variable from multicollinearity calculation")
    
    correlation_matrix = df[numeric_available_features].corr().abs()
    upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool_))
    high_corr_pairs = [(col1, col2) for col1 in upper_tri.columns for col2 in upper_tri.index if upper_tri.loc[col2, col1] > 0.95]

    print(f"\nHigh correlation pairs (>0.95):")
    for col1, col2 in high_corr_pairs:
        print(f"{col1} - {col2}: {upper_tri.loc[col2, col1]:.4f}")
    
    # Optionally remove one of each highly correlated pair
    features_to_drop = []
    for col1, col2 in high_corr_pairs:
        # # Keep count_std__total_issues even if highly correlated with another feature
        # if col2 == 'count_std__total_issues':
        #     # Swap the pair to keep count_std__total_issues
        #     features_to_drop.append(col1)
        # elif col1 == 'count_std__total_issues':
        #     # Already in the order we want
        #     features_to_drop.append(col2)
        # else:
            # Default: Keep the first one, drop the second one
            if col2 not in features_to_drop:
                features_to_drop.append(col2)

    print(f"\nDropping features due to multicollinearity: {features_to_drop}")
else:
    print("No numeric features available for multicollinearity check")
    features_to_drop = []

final_features = [f for f in available_features if f not in features_to_drop]
print(f"Final feature count: {len(final_features)}")

# Final check to ensure we've dropped the specified fields
for field in fields_to_exclude:
    if field in final_features:
        final_features.remove(field)
        print(f"Final check: Removed '{field}' from final_features")
        
# Print confirmation of fields not in final features
print("\nFinal confirmation - excluded fields:")
for field in fields_to_exclude:
    present = field in final_features
    print(f"'{field}' is {'STILL in' if present else 'NOT in'} the final feature list")

# Check if count_std__total_issues made it to the final list
if 'count_std__total_issues' in final_features:
    print("\nSuccessfully kept 'count_std__total_issues' in the final feature list")
else:
    print("\n'count_std__total_issues' is not in the final feature list")
    # If it's in the dataframe but not in final features, this is concerning
    if 'count_std__total_issues' in df.columns:
        print("CAUTION: 'count_std__total_issues' exists in the dataset but was excluded from final features")

# Prepare the data
X = df[final_features].copy()
y = df[target_variable]  # The target variable

# Fill any remaining NaN values with median
for col in X.columns:
    X[col] = X[col].fillna(X[col].median())

# Split the data with stratification on binned target to ensure similar distributions
y_binned = pd.qcut(y, q=10, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y_binned)

# Final verification: ensure target is not used as a feature
print("\nFinal verification before model training:")
print(f"Target variable: {target_variable}")
print(f"Number of features: {len(final_features)}")
if target_variable in final_features:
    print(f"ERROR: Target variable found in feature list!")
    final_features.remove(target_variable)
    X = df[final_features].copy()  # Recreate X without the target
    print(f"Removed target from features. New feature count: {len(final_features)}")
    
    # Re-split with corrected features
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y_binned)
else:
    print("Verified: Target is not being used as a feature")

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save data splits for later reference
with open(f'{output_dir}/data_splits.pkl', 'wb') as f:
    pickle.dump({
        'X_train': X_train,
        'X_test': X_test,
        'y_train': y_train,
        'y_test': y_test,
        'feature_names': final_features,
        'scaler': scaler
    }, f)

# Define the original models with more conservative hyperparameters to prevent overfitting
original_models = {
    'Random_Forest': RandomForestRegressor(
        n_estimators=100, 
        max_depth=10,
        min_samples_leaf=5,
        max_features='sqrt',
        random_state=42
    ),
    'Gradient_Boosting': GradientBoostingRegressor(
        n_estimators=100, 
        max_depth=5,
        learning_rate=0.05,
        min_samples_leaf=5,
        random_state=42
    ),
    'XGBoost': xgb.XGBRegressor(
        n_estimators=100, 
        max_depth=5, 
        learning_rate=0.05,
        min_child_weight=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
}

# Copy for tuning later
models = original_models.copy()

# Save original model hyperparameters for reference
original_params = {name: model.get_params() for name, model in original_models.items()}
with open(f'{output_dir}/original_hyperparameters.pkl', 'wb') as f:
    pickle.dump(original_params, f)

# First, train and evaluate original models
original_results = {}
original_predictions = {}

print("\nTraining original models (before hyperparameter tuning)...")
for name, model in original_models.items():
    print(f"\nTraining original {name}...")
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    original_predictions[name] = y_pred
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    spearman_corr, _ = spearmanr(y_test, y_pred)  # Rank correlation, less affected by outliers
    
    original_results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Spearman': spearman_corr
    }
    
    print(f"Original {name} - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}, Spearman: {spearman_corr:.4f}")
    
    # Save the original model
    with open(f'{output_dir}/original_{name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)

# Create original ensemble prediction
original_ensemble_pred = np.mean([original_predictions[name] for name in original_models.keys()], axis=0)

# Calculate metrics for original ensemble
original_ensemble_mae = mean_absolute_error(y_test, original_ensemble_pred)
original_ensemble_rmse = np.sqrt(mean_squared_error(y_test, original_ensemble_pred))
original_ensemble_r2 = r2_score(y_test, original_ensemble_pred)
original_ensemble_spearman, _ = spearmanr(y_test, original_ensemble_pred)

original_results['Ensemble'] = {
    'MAE': original_ensemble_mae,
    'RMSE': original_ensemble_rmse,
    'R2': original_ensemble_r2,
    'Spearman': original_ensemble_spearman
}

print(f"Original Ensemble - MAE: {original_ensemble_mae:.2f}, RMSE: {original_ensemble_rmse:.2f}, R2: {original_ensemble_r2:.4f}, Spearman: {original_ensemble_spearman:.4f}")

# Save original results
with open(f'{output_dir}/original_results.pkl', 'wb') as f:
    pickle.dump(original_results, f)

# Create prediction and residual plots for original models
for name, y_pred in original_predictions.items():
    print(f"\nCreating plots for original {name} model...")
    
    # Actual vs Predicted Plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([0, y_test.max()], [0, y_test.max()], 'r--')
    plt.xlabel('Actual Hours')
    plt.ylabel('Predicted Hours')
    plt.title(f'Original {name} - Actual vs Predicted')
    plt.savefig(f'{output_dir}/original_{name}_predictions.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Residual Plot
    plt.figure(figsize=(10, 6))
    residuals = y_test - y_pred
    plt.scatter(y_pred, residuals, alpha=0.3)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Hours')
    plt.ylabel('Residuals (Actual - Predicted)')
    plt.title(f'Original {name} - Residual Plot')
    plt.savefig(f'{output_dir}/original_{name}_residuals.png', dpi=300, bbox_inches='tight')
    plt.close()

# Create a plot for the original ensemble predictions
# Actual vs Predicted Plot for Original Ensemble
plt.figure(figsize=(10, 6))
plt.scatter(y_test, original_ensemble_pred, alpha=0.3)
plt.plot([0, y_test.max()], [0, y_test.max()], 'r--')
plt.xlabel('Actual Hours')
plt.ylabel('Predicted Hours')
plt.title('Original Ensemble - Actual vs Predicted')
plt.savefig(f'{output_dir}/original_ensemble_predictions.png', dpi=300, bbox_inches='tight')
plt.close()

# Residual Plot for Original Ensemble
plt.figure(figsize=(10, 6))
original_ensemble_residuals = y_test - original_ensemble_pred
plt.scatter(original_ensemble_pred, original_ensemble_residuals, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Hours')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Original Ensemble - Residual Plot')
plt.savefig(f'{output_dir}/original_ensemble_residuals.png', dpi=300, bbox_inches='tight')
plt.close()

print("\nGenerated prediction and residual plots for original models and ensemble.")

#########################
# Hyperparameter Tuning #
#########################

print("\nPerforming hyperparameter tuning...")

# Define parameter grids for each model
param_grids = {
    'Random_Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, None],
        'min_samples_leaf': [1, 3, 5],
        'max_features': ['sqrt', 'log2']
    },
    'Gradient_Boosting': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'min_samples_leaf': [1, 3, 5],
        'subsample': [0.8, 1.0]
    },
    'XGBoost': {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 3, 5],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Perform hyperparameter tuning for each model
tuned_models = {}
best_params = {}
best_scores = {}

for name, model in models.items():
    print(f"\nTuning {name}...")
    
    # Use RandomizedSearchCV for faster execution
    grid_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grids[name],
        n_iter=10,  # Try 10 random combinations
        cv=3,        # 3-fold cross-validation for speed
        scoring='neg_mean_absolute_error',  # Optimize for MAE
        n_jobs=-1,   # Use all available processors
        random_state=42,
        verbose=1
    )
    
    # Fit the grid search
    grid_search.fit(X_train_scaled, y_train)
    
    # Print best parameters
    print(f"Best parameters for {name}:")
    print(grid_search.best_params_)
    print(f"Best score: {-grid_search.best_score_:.4f} (MAE)")
    
    # Store best parameters and scores
    best_params[name] = grid_search.best_params_
    best_scores[name] = -grid_search.best_score_
    
    # Update the model with the best estimator
    tuned_models[name] = grid_search.best_estimator_

# Save hyperparameter tuning results
with open(f'{output_dir}/hyperparameter_tuning_results.pkl', 'wb') as f:
    pickle.dump({
        'best_params': best_params,
        'best_scores': best_scores
    }, f)

# Replace original models with tuned models for future steps
models = tuned_models

# Perform cross-validation on tuned models
print("\nPerforming 5-fold cross-validation with tuned models...")
tuned_cv_results = {}
for name, model in models.items():
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='r2')
    tuned_cv_results[name] = {
        'scores': cv_scores,
        'mean': cv_scores.mean(),
        'std': cv_scores.std()
    }
    print(f"{name} CV R² scores: {cv_scores}")
    print(f"{name} CV R² mean: {cv_scores.mean():.4f}, std: {cv_scores.std():.4f}")

# Save cross-validation results
with open(f'{output_dir}/tuned_cv_results.pkl', 'wb') as f:
    pickle.dump(tuned_cv_results, f)

# Train and evaluate tuned models
tuned_results = {}
tuned_predictions = {}

for name, model in models.items():
    print(f"\nTraining tuned {name}...")
    model.fit(X_train_scaled, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test_scaled)
    tuned_predictions[name] = y_pred
    
    # Calculate metrics
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    spearman_corr, _ = spearmanr(y_test, y_pred)
    
    tuned_results[name] = {
        'MAE': mae,
        'RMSE': rmse,
        'R2': r2,
        'Spearman': spearman_corr
    }
    
    print(f"Tuned {name} - MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.4f}, Spearman: {spearman_corr:.4f}")
    
    # Save the tuned model
    with open(f'{output_dir}/tuned_{name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)
    
    # Plot actual vs predicted
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.3)
    plt.plot([0, y_test.max()], [0, y_test.max()], 'r--')
    plt.xlabel('Actual Hours')
    plt.ylabel('Predicted Hours')
    plt.title(f'Tuned {name} - Actual vs Predicted')
    plt.savefig(f'{output_dir}/tuned_{name}_predictions.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Add residual subplot
    plt.figure(figsize=(10, 6))
    residuals = y_test - y_pred
    plt.scatter(y_pred, residuals, alpha=0.3)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicted Hours')
    plt.ylabel('Residuals (Actual - Predicted)')
    plt.title(f'Tuned {name} - Residual Plot')
    plt.savefig(f'{output_dir}/tuned_{name}_residuals.png', dpi=300, bbox_inches='tight')
    plt.close()

# Create a tuned ensemble prediction
tuned_ensemble_pred = np.mean([tuned_predictions[name] for name in models.keys()], axis=0)

# Calculate metrics for tuned ensemble
tuned_ensemble_mae = mean_absolute_error(y_test, tuned_ensemble_pred)
tuned_ensemble_rmse = np.sqrt(mean_squared_error(y_test, tuned_ensemble_pred))
tuned_ensemble_r2 = r2_score(y_test, tuned_ensemble_pred)
tuned_ensemble_spearman, _ = spearmanr(y_test, tuned_ensemble_pred)

tuned_results['Ensemble'] = {
    'MAE': tuned_ensemble_mae,
    'RMSE': tuned_ensemble_rmse,
    'R2': tuned_ensemble_r2,
    'Spearman': tuned_ensemble_spearman
}

print(f"Tuned Ensemble - MAE: {tuned_ensemble_mae:.2f}, RMSE: {tuned_ensemble_rmse:.2f}, R2: {tuned_ensemble_r2:.4f}, Spearman: {tuned_ensemble_spearman:.4f}")

# Save tuned results
with open(f'{output_dir}/tuned_results.pkl', 'wb') as f:
    pickle.dump(tuned_results, f)

# Plot actual vs predicted for tuned ensemble
plt.figure(figsize=(10, 6))
plt.scatter(y_test, tuned_ensemble_pred, alpha=0.3)
plt.plot([0, y_test.max()], [0, y_test.max()], 'r--')
plt.xlabel('Actual Hours')
plt.ylabel('Predicted Hours')
plt.title('Tuned Ensemble - Actual vs Predicted')
plt.savefig(f'{output_dir}/tuned_ensemble_predictions.png', dpi=300, bbox_inches='tight')
plt.close()

# Plot residuals for tuned ensemble
plt.figure(figsize=(10, 6))
tuned_ensemble_residuals = y_test - tuned_ensemble_pred
plt.scatter(tuned_ensemble_pred, tuned_ensemble_residuals, alpha=0.3)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Hours')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Tuned Ensemble - Residual Plot')
plt.savefig(f'{output_dir}/tuned_ensemble_residuals.png', dpi=300, bbox_inches='tight')
plt.close()

# Identify the best tuned model
best_tuned_model_name = min(tuned_results, key=lambda x: tuned_results[x]['MAE'] if x != 'Ensemble' else float('inf'))
best_tuned_model = models[best_tuned_model_name]
print(f"Best tuned model based on MAE: {best_tuned_model_name}")

# Save the best tuned model
with open(f'{output_dir}/best_tuned_model.pkl', 'wb') as f:
    pickle.dump(best_tuned_model, f)

# Plot feature importance for the best tuned model
if hasattr(best_tuned_model, 'feature_importances_'):
    # Get feature importances
    importances = best_tuned_model.feature_importances_
    
    # Sort features by importance
    indices = np.argsort(importances)[::-1]
    
    # Plot the top 20 features
    plt.figure(figsize=(12, 8))
    plt.title(f'Feature Importance - Best Tuned Model ({best_tuned_model_name})')
    plt.bar(range(min(20, len(final_features))), 
            importances[indices[:20]], 
            align='center')
    plt.xticks(range(min(20, len(final_features))), 
               [final_features[i] for i in indices[:20]], 
               rotation=90)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/best_tuned_model_feature_importance.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Create a DataFrame for easier analysis
    feature_importance_df = pd.DataFrame({
        'Feature': final_features,
        'Importance': importances
    }).sort_values('Importance', ascending=False)
    
    # Save feature importance to CSV
    feature_importance_df.to_csv(f'{output_dir}/feature_importance.csv', index=False)
    
    print("Top 10 most important features:")
    print(feature_importance_df.head(10))

    # Calculate and plot correlations between top features and the target
    top_features = [f for f in feature_importance_df['Feature'].head(10).tolist() if f in numeric_features]
    if top_features:
        # Final check that we don't include the target in the correlation matrix
        if target_variable in top_features:
            top_features.remove(target_variable)
            print(f"WARNING: Removed target from top features correlation")
            
        print(f"Calculating correlations for top {len(top_features)} numeric features")
        if top_features:
            corr_columns = top_features + [target_variable]
            corr_data = df[corr_columns].copy()
            
            plt.figure(figsize=(12, 10))
            sns.heatmap(corr_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
            plt.title('Correlation Between Top Features and Target')
            plt.tight_layout()
            plt.savefig(f'{output_dir}/top_features_correlation.png', dpi=300, bbox_inches='tight')
            plt.close()
        else:
            print("No valid top features remain for correlation analysis")
    else:
        print("No numeric top features available for correlation analysis")

    # Group feature importance by categories
    # Define categories based on feature name prefixes
    categories = {
        'Issue Type': [f for f in final_features if 'type_' in f or 'is_type_' in f],
        'Priority': [f for f in final_features if 'priority' in f],
        'Project Stats': [f for f in final_features if 'count_' in f or 'pct_' in f],
        'Team Size': [f for f in final_features if 'team_size' in f],
        'Created Time': [f for f in final_features if 'created_' in f],
        'Robust Stats': [f for f in final_features if 'stat_robust' in f],
        'Other': []
    }
    
    # Assign remaining features to 'Other'
    for feature in final_features:
        if not any(feature in cat_features for cat_features in categories.values()):
            categories['Other'].append(feature)
    
    # Calculate importance by category
    category_importance = {}
    for category, cat_features in categories.items():
        if cat_features:  # Skip empty categories
            total_importance = sum(feature_importance_df.loc[feature_importance_df['Feature'].isin(cat_features), 'Importance'])
            category_importance[category] = total_importance
    
    # Plot category importance
    plt.figure(figsize=(10, 6))
    categories_sorted = sorted(category_importance.items(), key=lambda x: x[1], reverse=True)
    categories_names = [item[0] for item in categories_sorted]
    categories_values = [item[1] for item in categories_sorted]
    
    plt.bar(categories_names, categories_values)
    plt.xlabel('Feature Category')
    plt.ylabel('Total Importance')
    plt.title('Feature Importance by Category')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/best_tuned_model_feature_importance_by_category.png', dpi=300, bbox_inches='tight')
    plt.close()

# Create visualizations to compare original vs tuned models
comparison_dir = f'{output_dir}/comparison'
os.makedirs(comparison_dir, exist_ok=True)

print("\nCreating model comparison visualizations...")

# Create comparison dataframe
comparison_data = []
for model_name in original_results.keys():
    if model_name != 'Ensemble' or (model_name == 'Ensemble' and model_name in tuned_results):
        for metric_name in ['MAE', 'RMSE', 'R2', 'Spearman']:
            original_value = original_results[model_name].get(metric_name, None)
            tuned_value = tuned_results.get(model_name, {}).get(metric_name, None)
            
            if original_value is not None and tuned_value is not None:
                # Calculate improvement
                if metric_name in ['MAE', 'RMSE']:  # Lower is better
                    improvement = ((original_value - tuned_value) / original_value) * 100
                else:  # Higher is better
                    improvement = ((tuned_value - original_value) / abs(original_value)) * 100
                
                comparison_data.append({
                    'Model': model_name,
                    'Metric': metric_name,
                    'Original': original_value,
                    'Tuned': tuned_value,
                    'Improvement (%)': improvement
                })

comparison_df = pd.DataFrame(comparison_data)
comparison_df.to_csv(f'{comparison_dir}/model_comparison.csv', index=False)

# Create a performance comparison bar chart for each metric
for metric in ['MAE', 'RMSE', 'R2', 'Spearman']:
    metric_data = comparison_df[comparison_df['Metric'] == metric]
    
    plt.figure(figsize=(12, 6))
    
    # Adjust these values based on your metric
    better_direction = 'lower' if metric in ['MAE', 'RMSE'] else 'higher'
    
    # Setup the bar chart
    bar_width = 0.35
    x = np.arange(len(metric_data['Model'].unique()))
    
    # Plot bars
    plt.bar(x - bar_width/2, metric_data['Original'], width=bar_width, label='Original', color='lightblue')
    plt.bar(x + bar_width/2, metric_data['Tuned'], width=bar_width, label='Hyperparameter Tuned', color='darkblue')
    
    # Add value labels
    for i, original in enumerate(metric_data['Original']):
        plt.text(i - bar_width/2, original * 1.02, f'{original:.4f}', ha='center', va='bottom', fontsize=9)
        
    for i, tuned in enumerate(metric_data['Tuned']):
        plt.text(i + bar_width/2, tuned * 1.02, f'{tuned:.4f}', ha='center', va='bottom', fontsize=9)
    
    # Customize the plot
    plt.xlabel('Model')
    plt.ylabel(metric)
    plt.title(f'{metric} Comparison: Original vs Tuned Models ({better_direction} is better)')
    plt.xticks(x, metric_data['Model'])
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig(f'{comparison_dir}/{metric}_comparison.png', dpi=300, bbox_inches='tight')
    plt.close()

# Create an improvement percentage chart
plt.figure(figsize=(12, 8))

# Pivot the data for easier plotting
pivot_df = pd.pivot_table(
    comparison_df, 
    values='Improvement (%)', 
    index='Model', 
    columns='Metric'
)

# Plot the improvement heatmap
sns.heatmap(
    pivot_df, 
    annot=True, 
    cmap='RdYlGn', 
    fmt='.1f',
    linewidths=.5, 
    center=0
)
plt.title('Percentage Improvement from Hyperparameter Tuning')
plt.tight_layout()
plt.savefig(f'{comparison_dir}/improvement_heatmap.png', dpi=300, bbox_inches='tight')
plt.close()

# Create side-by-side comparison of best models
# Identify best original and tuned models
best_original_model_name = min(original_results, key=lambda x: original_results[x]['MAE'] if x != 'Ensemble' else float('inf'))
best_tuned_model_name = min(tuned_results, key=lambda x: tuned_results[x]['MAE'] if x != 'Ensemble' else float('inf'))

print(f"Best original model: {best_original_model_name}")
print(f"Best tuned model: {best_tuned_model_name}")

# Create side-by-side comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Original model plot
ax1.scatter(y_test, original_predictions[best_original_model_name], alpha=0.3)
ax1.plot([0, y_test.max()], [0, y_test.max()], 'r--')
ax1.set_xlabel('Actual Hours')
ax1.set_ylabel('Predicted Hours')
ax1.set_title(f'Best Original Model: {best_original_model_name}')
ax1.grid(True, linestyle='--', alpha=0.7)

# Add metrics annotation
orig_metrics = original_results[best_original_model_name]
orig_metrics_text = f"MAE: {orig_metrics['MAE']:.2f}\nRMSE: {orig_metrics['RMSE']:.2f}\nR²: {orig_metrics['R2']:.4f}"
ax1.annotate(
    orig_metrics_text, 
    xy=(0.05, 0.95), 
    xycoords='axes fraction', 
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
    va='top'
)

# Tuned model plot
ax2.scatter(y_test, tuned_predictions[best_tuned_model_name], alpha=0.3)
ax2.plot([0, y_test.max()], [0, y_test.max()], 'r--')
ax2.set_xlabel('Actual Hours')
ax2.set_ylabel('Predicted Hours')
ax2.set_title(f'Best Tuned Model: {best_tuned_model_name}')
ax2.grid(True, linestyle='--', alpha=0.7)

# Add metrics annotation
tuned_metrics = tuned_results[best_tuned_model_name]
tuned_metrics_text = f"MAE: {tuned_metrics['MAE']:.2f}\nRMSE: {tuned_metrics['RMSE']:.2f}\nR²: {tuned_metrics['R2']:.4f}"
ax2.annotate(
    tuned_metrics_text, 
    xy=(0.05, 0.95), 
    xycoords='axes fraction', 
    bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8),
    va='top'
)

plt.suptitle('Best Models Comparison: Original vs Tuned', fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.savefig(f'{comparison_dir}/best_models_comparison.png', dpi=300, bbox_inches='tight')
plt.close()

# Generate summary report
with open(f'{output_dir}/model_summary.txt', 'w') as f:
    f.write("# Resolution Hours Ensemble Model with Hyperparameter Tuning\n\n")
    f.write("## Performance Comparison\n\n")
    
    # Write performance table
    f.write("| Model | Metric | Original | Tuned | Improvement (%) |\n")
    f.write("|-------|--------|----------|-------|----------------|\n")
    
    for _, row in comparison_df.iterrows():
        f.write(f"| {row['Model']} | {row['Metric']} | {row['Original']:.4f} | {row['Tuned']:.4f} | {row['Improvement (%)']:.2f} |\n")
    
    # Write hyperparameter differences
    f.write("\n## Hyperparameter Changes\n\n")
    for model_name in best_params:
        f.write(f"### {model_name}\n\n")
        f.write("| Parameter | Original | Tuned |\n")
        f.write("|-----------|----------|-------|\n")
        
        orig_params = original_params[model_name]
        tuned_params = best_params[model_name]
        
        for param, tuned_value in tuned_params.items():
            orig_value = orig_params.get(param, "N/A")
            f.write(f"| {param} | {orig_value} | {tuned_value} |\n")
        
        f.write("\n")
    
    # Write conclusion
    f.write("\n## Conclusion\n\n")
    
    # Calculate average improvement
    mae_improvement = comparison_df[comparison_df['Metric'] == 'MAE']['Improvement (%)'].mean()
    r2_improvement = comparison_df[comparison_df['Metric'] == 'R2']['Improvement (%)'].mean()
    
    f.write(f"Hyperparameter tuning resulted in an average MAE improvement of {mae_improvement:.2f}% ")
    f.write(f"and an average R² improvement of {r2_improvement:.2f}%.\n\n")
    
    if best_original_model_name == best_tuned_model_name:
        f.write(f"Both before and after tuning, {best_original_model_name} was the best performing model.\n")
    else:
        f.write(f"Before tuning, {best_original_model_name} was the best performing model. ")
        f.write(f"After tuning, {best_tuned_model_name} became the best performing model.\n")

print(f"\nModel training, tuning, and comparison complete. Results saved to {output_dir}/")
print(f"Generated model comparison visualizations in {comparison_dir}/")
print(f"See {output_dir}/model_summary.txt for a detailed report")

# If count_std__total_issues was kept, check its importance
if 'count_std__total_issues' in final_features and hasattr(best_tuned_model, 'feature_importances_'):
    count_feature_idx = final_features.index('count_std__total_issues')
    count_feature_importance = importances[count_feature_idx]
    count_feature_rank = list(indices).index(count_feature_idx) + 1
    
    print(f"\nImportance of 'count_std__total_issues':")
    print(f"- Importance value: {count_feature_importance:.6f}")
    print(f"- Rank among all features: {count_feature_rank} out of {len(final_features)}")
    print(f"- Percentile: {100 - (count_feature_rank / len(final_features) * 100):.1f}%")
    
    # Highlight this in the summary as well
    with open(f'{output_dir}/model_summary.txt', 'a') as f:
        f.write("\n## Special Feature Analysis: count_std__total_issues\n\n")
        f.write(f"Importance value: {count_feature_importance:.6f}\n")
        f.write(f"Rank among all features: {count_feature_rank} out of {len(final_features)}\n")
        f.write(f"Percentile: {100 - (count_feature_rank / len(final_features) * 100):.1f}%\n")
        
        if count_feature_rank <= 10:
            f.write("\nThis feature is among the top 10 most important features for the model.\n")
        elif count_feature_rank <= len(final_features) // 4:
            f.write("\nThis feature is in the top 25% of all features by importance.\n")
        else:
            f.write("\nThis feature has moderate importance in the model.\n")

# Print final message about count_std__total_issues to guide further analysis
if 'count_std__total_issues' in final_features:
    print("\nNOTE: The 'count_std__total_issues' feature was successfully included in the model.")
    print("Check the feature importance section in the summary report to see its impact.")
else:
    print("\nNOTE: The 'count_std__total_issues' feature was not found in the final feature list.")
    print("Possible reasons:")
    print("1. It doesn't exist in the original dataset")
    print("2. It was removed during preprocessing")
    print("3. It was highly correlated with another feature and removed during multicollinearity check")
    
print("\nAnalysis complete! The hypertuned model is ready for use.")

Original dataset shape: (1575833, 63)
Reduced dataset shape (5%): (787916, 63)

Fields to exclude that exist in the dataframe: ['fields.status.name', 'fields.project.key', 'fields.priority.name', 'fields.project.name']
Dropping these fields from the dataframe: ['fields.status.name', 'fields.project.key', 'fields.priority.name', 'fields.project.name']
Dataframe shape after dropping: (787916, 59)
Missing values in target: 0
Dataset shape after dropping missing targets: (787916, 59)

Using 57 numeric features for correlation calculation
Top 10 correlated features with target:
count_std__total_issues                 0.862686
remainder__team_size_combined           0.756612
remainder__team_size_creators           0.756612
time_power__project_duration_days       0.431717
stat_robust__bug_ratio                  0.294738
pct_minmax__type_bug_pct                0.264221
avg_resolution_hours                    0.258149
stat_robust__weighted_priority_score    0.186916
age_days                    